In [495]:
from typing import TypeVar, Tuple,  List, Iterator, Optional
import json
import string
import math

## PriorityQueue
PriorityQueue references heapq and returns the location of the grid with the lowest priority each time

In [496]:
import heapq
T = TypeVar('T')
class PriorityQueue:
    def __init__(self):
        self.elements: list[tuple[float, T]] = []

    def empty(self) -> bool:
        return  not self.elements

    def put(self, item: T, priority: float):
        heapq.heappush(self.elements, (priority, item))

    def get(self) -> T:
        return heapq.heappop(self.elements)[1]

## Graph

In [497]:
GridLocation = Tuple[int, int]
class Graph:
    def __init__(self, graph_data: List[ List[int] ], heuristic_type: int):
        # Remember the graph_data and heuristic
        self.graph_data = graph_data
        self.heuristic_type = heuristic_type
        self.width = len(graph_data[0])
        self.height = len(graph_data)

        self.maxheight = self.graph_data[0][0]
        self.minheight = self.graph_data[0][0]
        for i in range(self.height):
            for j in range(self.width):
                self.maxheight = max(self.maxheight, self.graph_data[i][j])
                self.minheight = min(self.minheight, self.graph_data[i][j])


    def in_bounds(self, id: GridLocation) -> bool:
        (x, y) = id
        return 0 <= x < self.width and 0 <= y < self.height


    def neighbours(self, id: GridLocation) -> Iterator[GridLocation]:
        (x, y) = id
        neighbors = [(x + 1, y), (x, y + 1), (x - 1, y), (x, y - 1)]
        return filter(self.in_bounds, neighbors)

    def edge_length(self, from_id: GridLocation, to_id: GridLocation) -> float:
        (x1, y1) = from_id
        (x2, y2) = to_id
        return 1 + (self.graph_data[x1][y1] - self.graph_data[x2][y2])*(self.graph_data[x1][y1] - self.graph_data[x2][y2])


    def heuristic(self, from_id: GridLocation, to_id: GridLocation) -> float:
        (x1, y1) = from_id
        (x2, y2) = to_id

        match self.heuristic_type:
            case "0":
                # Actually, A* turns into Dijkstra’s Algorithm
                return 0
            case "1":
                # “as the crow flies"
                return math.sqrt( abs(x1 - x2) * abs(x1 - x2) + abs(y1 - y2) * abs(y1 - y2))
            case "2":
                # Manhattan distance on a square grid
                return abs(x1 - x2) + abs(y1 - y2)
            case "3":
                # Actually, A* turns into Greedy Best-First-Search
                return 1000*(abs(x1 - x2) + abs(y1 - y2))
            case "4":
                mincost: float = abs(x1 - x2) + abs(y1 - y2);
                maxcost: float = (abs(x1 - x2) + abs(y1 - y2)) * (1 + (self.maxheight - self.minheight) * (self.maxheight - self.minheight));
                return (100 * 1.0 / 1000)*(maxcost - mincost) + mincost
            case _:
                print("other case")

## reconstruct_path

In [498]:
def reconstruct_path(came_from: dict[GridLocation, Optional[GridLocation]], start_node: GridLocation, end_node: GridLocation) -> List[GridLocation]:
    path: List[GridLocation] =[]

    if end_node not in came_from:
        print("there is no path exits !")
        return []

    current: GridLocation = end_node

    while current != start_node:
        path.append(current)
        current = came_from[current]

    path.append(start_node)
    path.reverse()

    return path



## find_shortest_path
using $A^* search$

In [499]:
def find_shortest_path(graph: Graph, start_node: GridLocation, end_node: GridLocation):
    frontier = PriorityQueue()
    frontier.put(start_node, 0)
    # Since dict[star_tnode] = None, we have to add "Optional"
    came_from: dict[GridLocation, Optional[GridLocation]] = {}
    cost_so_far: dict[GridLocation, float] = {}
    came_from[start_node] = None
    cost_so_far[start_node] = 0

    while not frontier.empty():
        current: GridLocation = frontier.get()
        if current == end_node:
            break
        for next in graph.neighbours(current):
            new_cost: float = cost_so_far[current] + graph.edge_length(current,next)
            if next not in cost_so_far or new_cost < cost_so_far[next]:
                cost_so_far[next] = new_cost
                priority: float = new_cost + graph.heuristic(next, end_node)
                frontier.put(next, priority)
                came_from[next] = current

    path = reconstruct_path(came_from, start_node, end_node)

    return {
                'path': path,
                'vertex': len(path),
                'explored': len(came_from),
                "distance": cost_so_far[end_node]
           }

## print_path

In [500]:
# Function to print the path.
# Uses letters a-z for height, A-Z for height on the path.
def print_path(data, path):
    for i in range(len(data)):
        for j in range(len(data[i])):
            if (i,j) in path:
                print(string.ascii_uppercase[data[i][j]],end="")
            else:
                print(string.ascii_lowercase[data[i][j]],end="")
        print("")

In [501]:
# Read problem file
filename = input("Enter problem filename:")
with open(filename) as f:
    data = json.load(f)

heuristic_type = input("Enter heuristic type: ")

# Go from top-left to bottom-right
startnode = (0,0)
endnode = (len(data)-1, len(data[0])-1)
graph = Graph(data, heuristic_type)

output = find_shortest_path(graph, startnode, endnode)

print("")
print("----------------------")
print(f"Search explored {output['explored']} nodes")
print(f"The path pass {output['vertex']} vertex")
print(f"The shortest path is length: {output['distance']}")
print(f"The path is: {output['path']}")


print_path(data, output['path'])



----------------------
Search explored 77 nodes
The path pass 31 vertex
The shortest path is length: 704
The path is: [(0, 0), (1, 0), (2, 0), (3, 0), (3, 1), (3, 2), (3, 3), (4, 3), (5, 3), (5, 4), (5, 5), (6, 5), (6, 6), (6, 7), (5, 7), (5, 8), (5, 9), (5, 10), (6, 10), (7, 10), (7, 11), (7, 12), (8, 12), (8, 13), (9, 13), (10, 13), (11, 13), (12, 13), (12, 14), (13, 14), (14, 14)]
Bhstkgicfjimssr
Enmjfkrnaamfmoj
Krqibbecbfrriqg
HEFGlphtqlsrsmj
trrEhnbnldombml
edpCBEtJPNHrenr
tlhdnEEBsbLsiar
moiiktqqbkNHBcr
piilcqjkfabqENa
pidgiibheatooGf
tooepeosgmglaLn
ctsrntmtnlgaiNc
jmsifjbbnseobHD
ndhkekrnlljttbK
lpblqaghcqccmsB
